# Análisis de Texto



Mediante el análisis del contenido de los tweets podemos obtener información adicional, por ejemplo, los temas que se está tratando, si se comparten páginas web, etc. Existen módulos y herramientas especializadas para realizar análisis de texto, sin embargo en este curso sólo utilizaremos las herramientas básicas de python para comparar cadenas de caracteres.

Si hemos seguido el notebook hasta este punto, debemos tener en la carpeta _data_ el archivo ¨stream_lunch.json¨, el cual contiene un conjunto de tweets obtenidos a traves del stremming. Uno de los procesos que facilita análisi texto es sustituir los caracteres con acento y signos de interrogación y admiración de apertura ("¿","¡") por caracteres sin acento y sólo signos de cierre "?" y "!". Además, podemos remover ligas a páginas web para que las palabras en ellas no interfieran en el análisis y reemplazar mayúsculas a minúsculas. Esto lo hacemos porque no todos los usuarios incluyen acentos, signos iniciales "¿","¡", páginas o mayúsculas.

Comencemos extrayendo el texto de todos los tweets, sustituyendo caractéres y guardándolo en un archivo independiente para facilitar una posible inspección visual. El siguiente código extraemos tweets con un cierto tema y ademas guarda en memoria una lista `Textos` con el contenido de cada tweet:

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener

import json
import tweepy

consumer_key='XXXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret= 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

authentication = tweepy.OAuthHandler(consumer_key, consumer_secret)
authentication.set_access_token(access_token, access_secret)

api = tweepy.API(authentication)

with open("./data/stream_lunch.json","w") as archivo:
    pass
class TwitterListener(StreamListener):

    def on_data(self,data):
        tweet=json.loads(str(data))
        print(tweet['text'])
        with open("./data/stream_lunch.json","a") as archivo:
            archivo.write(json.dumps(tweet))
            archivo.write('\n')
        return(True)

    def on_error(self, status):
        print(status)

while True:
        try:
            twitter_stream = Stream(authentication, TwitterListener())
            twitter_stream.filter(track=['#FelizViernes'])
        except:
            pass

Ahora leemos los tweets almacenado en el archivo `stream_lunch.json` y guardamos los textos en una lista llamada `Textos`. 

In [ ]:
import json
from unidecode import unidecode

with open("./data/stream_lunch.json","r") as archivo:
    renglones=archivo.readlines()

Textos=[]
with open("./data/Textos.txt","w") as archivo2:
    for data in renglones:
        tuit=json.loads(data)
        texto=unidecode(tuit['text']) #
        texto.encode('ascii')# Reemplazan caracteres por su equivalente ascii
        Textos.append(texto)
        archivo2.write(texto)
        archivo2.write('\n\n')

Podemos ver el efecto de sustituir los caracteres acentuados imprimiendo el texto original y el modificado

In [ ]:
L=[json.loads(data)['text'] for data in renglones]
for i in range(13):
    print(L[i])
    print(Textos[i])
    print('\n\n')

Podemos remover los links que se comparten en el texto, por ejemplo:

In [ ]:
import re
for i in range(10):
    sin_url = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', Textos[i])
    print(sin_url)
    print('\n\n')

Repitamos el proceso de remover direcciones de páginas para todos los textos y los almacenémoslos de esta manera

In [ ]:
import re

Textos=[re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', texto) for texto in Textos]    

In [ ]:
print(Textos[:10])

Adicionalmente, resulta conveniente reemplazar todas las letras mayúsculas por minúsculas, pues recordemos que Python las interpreta como caracteres distintos, esto lo podemos hcer mediante el siguiente código:

In [ ]:
Textos=[texto.lower() for texto in Textos]

El contenido del texto lo analizaremos a partir de las palabras que aparecen en él, sin embargo no todas las palabras tienen la misma importancia, por ejemplo, los artículos, pronombres, preposiciones, etc no son palabras representativas del contenido, y por esta razón a estas palabras comúnmente se les denomina _palabras vacías_ ó _stop words_. En la carpeta _data_ se encuentra una lista de _palabras vacías_ que puede extenderse y adaptarse. El siguiente código lee y almacena en una lista las _palabras stop_ para posteriormente utilizarlas.

In [ ]:
#palabras_vacias='./data/stop_words.txt'
with open('./data/stopwordES.txt',"r") as archivo:
    palabras_stop=archivo.readlines()
    
palabras_stop=[palabra[:-1] for palabra in palabras_stop]
    

In [ ]:
print(palabras_stop)

Además de las palabras _vacías_ o _stop_, podemos crear listas adicionales con palabras asociadas a un tema; ya que si recolectamos los tweets buscando un _hashtag_ o una palabra en particular, tenemos una idea de qué palabras pueden estar presentes. Por ejemplo, si recolectamos tweets buscando la palabra "accidente", muchos de los tweets pueden contener además la palabra "automovil", "derrumbe", etc, y de igual manera evitar los tweets que contenga otras palabras relacionadas con otros temas. Por ejemplo, para ver cuántos de los tweets cuyo mensaje almacenamos en la variable ``Textos` contienen palabras asociadas a un tema, podemos usar el siguiente código:

In [ ]:
semantica=['mañana','dia','amanecer']
contador=0

for texto in Textos:
    for palabra in semantica:
        if palabra in texto:
             contador=contador+1
print(contador)

**Nota**: Recuerda que buscamos palabras con letras mayúsculas nunca las vamos a encontrar, pues hemos sustitutido letras mayúsculas por minúsculas

**Ejercicio:** Modifica las palabras a buscar de acuerdo a tus intereses.

Un análisis más fino requiere analizar palabra por palabra, así podríamos identificar las palabras de mayor frecuencia. Primero debemos dividir cada texto en palabras, y después almacenar las palabras que no sean _stop_ junto con el número de veces que es usada. Para agilizar este progeso, sólo compararemos palabras que no inicien con caracteres numéricos o caracteres _especiales_ ("@" , "#", "$","%"). Veamos un ejemplo.

In [ ]:
diccionario={}
#Creamos un diccionario de las palabras que no son stop
for texto in Textos:
    t=texto.split(' ') # separa por cadena de texto por palabras y las guarda en una lista t
    for palabra in t:
        if palabra != '':
            if (palabra not in palabras_stop) and (palabra[0] not in range(10)) and (palabra[0] not in ["@","#","$","%","/","\n","!","?"]) :
                diccionario[palabra]=0

            
#Corremos un codigo similar pero ahora realiza el conteo de las apariciones de las palabras en el diccionario
for texto in Textos:
    t=texto.split(' ') 
    for palabra in t:
        if palabra != '':
            if (palabra not in palabras_stop) and (palabra[0] not in range(10)) and (palabra[0] not in ["@","#","$","%","/","\n","!","?"]) :
                diccionario[palabra]=diccionario[palabra]+1

Inspeccionemos el diccionario buscando los valores de ocurrencia más altas:

In [ ]:
valores=[diccionario[valor] for valor in diccionario.keys()] # Lista los valores de cada palabra en el diccionario
valores.sort()  #Ordena la lista de menor a mayor 
valores.reverse() #invierte el orden de la lista 

print(valores[:10])  #imprime los 10 valores más altos

Ahora que sabemos el rango de ocurrencia de las palabras, busquemos en el diccionario aquellas palabras con un índice alto:

In [ ]:
alta_ocurr=[]

for palabra in diccionario:
    if diccionario[palabra]>= : #Debemos incluir un valor numerico por el valor en la frecuencia que nos interese encontrar
        alta_ocurr.append(palabra)
        
print(alta_ocurr)

Si nos interesan saber cuales son todas las palabras que encontramos solo debemos escribir lo siguiente:  

In [ ]:
print(diccionario.keys())